In [3]:
import pandas as pd
full_df = pd.read_csv('split_df.csv', sep = '|')
df2 = pd.read_csv('feats.csv')
df2 = df2.ix[:,1:3]

In [4]:
full_df = full_df.ix[:,0:26].merge(df2)

In [5]:
#full_df.loc[(full_df['Interest_Level']=='high')]
#filter(full_df, full_df.ix[:,'features'] != None)
#full_df[pd.isnull(full_df.features) == True]
#notfull_df.ix[92,'features']
full_df = full_df.fillna(value = 'No_Features')


In [6]:
"""import pandas as pd
full_df = pd.read_csv('full_df.csv', H)
df2 = pd.read_csv('feats.csv', sep = '|')
"""
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances


import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords

stop = stopwords.words('english')
def cleaning_feats(sentence):
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    #cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
    #selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned= cleaned.strip()
    return cleaned
	  
full_df['cln_feats'] = map(lambda x: cleaning_feats(x), full_df['features'])


data_high=full_df.loc[(full_df['interest_level']=='high')]
data_medium=full_df.loc[(full_df['interest_level']=='medium')]
data_low=full_df.loc[(full_df['interest_level']=='low')]

binVectorizer = CountVectorizer(binary=True)
counts = binVectorizer.fit_transform(full_df['cln_feats'])

In [7]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
targets = full_df['interest_level'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [8]:
examples = full_df['cln_feats']
example_counts = binVectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions_df=pd.DataFrame(predictions)
predictions_df.head(10)
actual=full_df['interest_level'].values
from sklearn.metrics import confusion_matrix
matrix=pd.DataFrame(confusion_matrix(actual, predictions,labels=["low", "medium", "high"]))
#print(matrix)

#pd.crosstab(full_df['Interest_Level'], predictions, rownames=['True'], colnames=['Predicted'], margins=True)
#predictions
import numpy as np
import pandas as pd
import scipy.sparse as sparse

full_df['Predicted_feats'] = predictions.tolist()


In [9]:
full_df['Predicted_feats']

0           low
1           low
2           low
3           low
4           low
5           low
6           low
7           low
8           low
9           low
10          low
11          low
12          low
13          low
14          low
15       medium
16          low
17          low
18          low
19          low
20       medium
21          low
22          low
23          low
24          low
25          low
26          low
27          low
28          low
29          low
          ...  
49322       low
49323       low
49324       low
49325       low
49326       low
49327       low
49328       low
49329       low
49330       low
49331       low
49332       low
49333       low
49334       low
49335       low
49336       low
49337       low
49338       low
49339       low
49340       low
49341      high
49342    medium
49343       low
49344       low
49345       low
49346       low
49347       low
49348       low
49349       low
49350       low
49351       low
Name: Predicted_feats, d

In [10]:
full_df.to_csv('classified_features.csv')